### Embeddings Extractor

In [1]:
import numpy as np
from torchvision.datasets import CIFAR100
from torchvision import transforms
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

You can swap the model for your prefered pretrained model.
I am using [SWAV](https://github.com/facebookresearch/swav), [paper](https://arxiv.org/pdf/2006.09882.pdf).

Which is unsupervised.

In [2]:
model = torch.hub.load('facebookresearch/swav', 'resnet50')
modules=list(model.children())[:-1]
model=nn.Sequential(*modules)
for p in model.parameters():
    p.requires_grad = False
model.cuda();

Using cache found in /home/dinari/.cache/torch/hub/facebookresearch_swav_master


In [3]:
tr_normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.228, 0.224, 0.225])
transforms = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            tr_normalize])
dataset = CIFAR100('../data/cifar100/',train=True, download=True, transform=transforms)

0it [00:00, ?it/s]

100%|█████████▉| 168665088/169001437 [00:33<00:00, 5222818.96it/s] 

Extracting ../data/cifar100/cifar-100-python.tar.gz to ../data/cifar100/


In [4]:
BATCH_SIZE = 8

In [5]:
loader = DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=1)

In [6]:
all_labels = []
all_embeddings = []
torch.cuda.empty_cache()
model.eval()
for data,labels in tqdm(loader):
    new_labels = labels.numpy().tolist()
    all_labels += new_labels
    data = data.cuda()
    embeddings = model(data.cuda())
    all_embeddings.append(np.reshape(embeddings.detach().cpu().numpy(),(len(new_labels),-1)))
all_embeddings = np.vstack(all_embeddings)


  4%|▎         | 223/6250 [00:09<04:17, 23.44it/s]
169009152it [00:50, 5222818.96it/s]                               
 14%|█▍        | 904/6250 [00:38<03:35, 24.79it/s]


 29%|██▊       | 1795/6250 [01:14<03:00, 24.73it/s]


 43%|████▎     | 2683/6250 [01:50<02:24, 24.76it/s]


 57%|█████▋    | 3571/6250 [02:26<01:48, 24.73it/s]


 71%|███████▏  | 4459/6250 [03:02<01:17, 23.23it/s]


 86%|████████▌ | 5347/6250 [03:40<00:38, 23.66it/s]


100%|█████████▉| 6235/6250 [04:17<00:00, 24.61it/s]


100%|██████████| 6250/6250 [04:18<00:00, 24.17it/s]


In [ ]:
DATA_PATH = '../data/cifar100/cifar100_embeddings.npy'
LABELS_PATH = '../data/cifar100/cifar100_labels.npy'

In [17]:
np.save(DATA_PATH,all_embeddings)
np.save(LABELS_PATH,np.array(all_labels))